#  전이학습 기반 분류기

In [1]:
# from PIL import Image
# a = np.array([[1,2,3],[1,2,3],[1,2,3]])
# img = Image.fromarray(a)
# b = np.array(img)

In [1]:
import scipy as sp
import numpy as np
import pandas as pd
from numpy.random import rand
pd.options.display.max_colwidth = 600


from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.model_selection import train_test_split

import cnn_utils as utils


import matplotlib.pyplot as plt
%matplotlib inline
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}

plt.rcParams.update(params)


from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
import keras
from tensorflow.keras import callbacks, optimizers,Model
from tensorflow.keras.applications import vgg16 as vgg
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


## 데이터 세트 로딩과 준비

In [17]:
# BATCH_SIZE = 64
# EPOCHS = 50    # 기본: 100
# NUM_CLASSES = 10 # 카테고리수 
# LEARNING_RATE = 1e-4
# MOMENTUM = 0.9
# PATH=r"C:\Users\admin\Desktop\image\#5"

In [3]:
BATCH_SIZE = 64
EPOCHS = 40    # 기본: 100
NUM_CLASSES = 8 # 카테고리수 
LEARNING_RATE = 1e-4
MOMENTUM = 0.9
PATH=r"C:\Users\admin\Desktop\image\#3"

In [4]:
base_model = vgg.VGG16(weights='imagenet', 
                       include_top=False, 
                       input_shape=(128, 128, 3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
# VGG16 모델의 세 번째 블록에서 마지막 층 추출
# VGG16 모델의 네 번째 블록에서 마지막 층 추출
last = base_model.get_layer('block4_pool').output
# last = base_model.get_layer('block3_pool').output

## 모델 준비

* 최상위층 없이 VGG16 로딩
* 커스텀 분류기 준비
* 모델의 맨 위에 새로운 층 쌓기

In [6]:
# 상위 층에 분류층 추가
x = GlobalAveragePooling2D()(last)
x= BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.7)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.6)(x)
pred = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(base_model.input, pred)


model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [7]:
checkpoint = keras.callbacks.ModelCheckpoint('model_cnn_#3-{epoch:03d}-{acc:03f}-{val_acc:03f}.h5', verbose=1,
                                             monitor='val_loss',
                                             save_best_only=True, mode='auto')

In [8]:
for layer in base_model.layers:
     layer.trainable = False

In [9]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=LEARNING_RATE),
              metrics=['accuracy'])

In [10]:
train_datagen = ImageDataGenerator(
                 rescale=1/255,
                validation_split=0.33
                                  )

In [11]:
train_generator = train_datagen.flow_from_directory(
        PATH,
        shuffle=True,
        seed=13,
        target_size=(128,128),
        batch_size=BATCH_SIZE,
        class_mode = 'categorical',
        subset="training")

Found 3905 images belonging to 8 classes.


In [12]:
val_generator = train_datagen.flow_from_directory(
        PATH,
        shuffle=True,
        seed=13,
        target_size=(128,128),
        batch_size=BATCH_SIZE,
        class_mode = 'categorical',
        subset="validation")


Found 1918 images belonging to 8 classes.


In [14]:
train_steps_per_epoch =100
val_steps_per_epoch = 100
path_csv = 'log_#3.csv'
from keras.callbacks import CSVLogger

csv_logger = CSVLogger(f'./{path_csv}', append=True, separator=';')


history = model.fit_generator(train_generator,
                              steps_per_epoch=train_steps_per_epoch,
                              validation_data=val_generator,
                              validation_steps=val_steps_per_epoch,
                              epochs=EPOCHS,
                              verbose=1, callbacks=[checkpoint,csv_logger])

Epoch 1/40
100/100 [==============================] - 229s 2s/step - loss: 0.3373 - acc: 0.8751

Epoch 00001: val_loss improved from inf to 0.33735, saving model to model_cnn_#3-001-0.841210-0.875059.h5
100/100 [==============================] - 451s 5s/step - loss: 0.4886 - acc: 0.8412 - val_loss: 0.3373 - val_acc: 0.8751
Epoch 2/40
100/100 [==============================] - 229s 2s/step - loss: 0.3241 - acc: 0.8753

Epoch 00002: val_loss improved from 0.33735 to 0.32410, saving model to model_cnn_#3-002-0.848351-0.875313.h5
100/100 [==============================] - 453s 5s/step - loss: 0.4476 - acc: 0.8484 - val_loss: 0.3241 - val_acc: 0.8753
Epoch 3/40
100/100 [==============================] - 228s 2s/step - loss: 0.3146 - acc: 0.8761

Epoch 00003: val_loss improved from 0.32410 to 0.31457, saving model to model_cnn_#3-003-0.854998-0.876114.h5
100/100 [==============================] - 452s 5s/step - loss: 0.4200 - acc: 0.8550 - val_loss: 0.3146 - val_acc: 0.8761
Epoch 4/40
100/10

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('VGG16_#3.png')
plt.show()

In [ ]:
# BATCH_SIZE = 64
# EPOCHS = 50    # 기본: 100
# NUM_CLASSES = 8 # 카테고리수 
# LEARNING_RATE = 1e-4
# MOMENTUM = 0.9
# PATH=r"C:\Users\admin\Desktop\image\#3"

In [ ]:
pip install graphviz
pip install pydot

In [16]:
model.save('./vgg16_model.h5')

In [17]:
plt.savefig('vgg16.png')

<Figure size 1080x360 with 0 Axes>

In [20]:
plt.show()